In [1]:
import numpy as np
import operator
from math import sqrt
from functools import partial
import sklearn
from sklearn import datasets
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import warnings
from sklearn.exceptions import ConvergenceWarning
"""
https://arxiv.org/pdf/1805.08125.pdf

Allocation Function. AF : (Pn, bn) → Sn, Sn ⊂ [M], the allocation function, takes as input the
price p_n and the bid from a buyer b_n, to return the features with noise added according to the bid.

Revenue Function. RF : (p_n, b_n, Y_n M, G) → r_n, r_n ∈ R+, the revenue function, takes as
input the current price p_n, the bid and the prediction task provided by the buyer (b_n and Y_n
respectively), to decide how much revenue r_n to extract from the buyer.

Payment Division Function. PD : (S_n, Y_n ;M, G) → ψ_n, ψ_n ∈ [0, 1]|S_n|
, the payment-division function, takes as input the set of features that were allocated XSn
along with the prediction task Y_n, to compute ψ_n, a vector denoting the marginal value 
of each allocated feature for the prediction task.

Price Update Function. PF : (Pn, bn, Yn) → Pn+1, Pn+1 ∈ RM
+ , the price-update function, takes
as input the current price vector Pn, the bid and the prediction task provided by the buyer (bn and Yn
respectively) to update the price vector for each of the M features.
"""

"""
Property A.1: Algorithm converges to the maximum price over time
Description of Experiment 1.1
Take n example datasets from Scikit-Learn [(X_i), Y_i]
[(X_1), (X_2), …, (X_n)] are features on sale
[Y_1, Y_2, …., Y_n] are prediction tasks “for hire”
[b_1, b_2, ….., b_n] are bids for each prediction type  
Dynamics
Uniformly at random select (Y_n, b_n) as stream of buyers
Desired outcomes of experiment  
(p_i) converge to approximately b_i 
Why? Optimal Outcome is that price of each (feature set type) converges to bid for that that prediction task type
Regularization helps
Why? - Theory shows that it makes the problem “convex”
"""
dset_names = [
 'load_boston',
 'load_breast_cancer',
 'load_diabetes',
 'load_digits',
]
    
dsets = {}
for attr in dset_names:
    name = attr.split('load_')[1]
    if name:
        dsets[name] = getattr(datasets, attr)()
Y_dict = { name: dsets[name].target for name in dsets }
X_dict = { name: dsets[name].data for name in dsets }

In [2]:
[(key, x.shape) for key, x in X_dict.items()]

[('boston', (506, 13)),
 ('breast_cancer', (569, 30)),
 ('diabetes', (442, 10)),
 ('digits', (1797, 64))]

In [3]:
[(key, y.shape) for key, y in Y_dict.items()]

[('boston', (506,)),
 ('breast_cancer', (569,)),
 ('diabetes', (442,)),
 ('digits', (1797,))]

In [4]:
""" Look at first four, up to 400 time steps (IMPORTANT: ml_algorithm is trained on all features available)"""
X_t1_raw = np.concatenate([x[:400] for x in X_dict.values()], axis=1)
Y_t1 = np.stack([y[:400] for y in Y_dict.values()])
# normalize each dataset to [0,1] in order to have added noise similarly affect each stream 
X_t1 = preprocessing.normalize(X_t1_raw, axis=0, norm='max')

In [5]:
X_t1_raw.shape, Y_t1.shape

((400, 117), (4, 400))

In [6]:
sum(x.shape[1] for key, x in X_dict.items()) # number of features

117

In [7]:
def get_original_columns(X_n, feature_idxs):
    indices = [idx for (idx, price) in feature_idxs]
    return X_n[:, indices]

def gain_score(y, x):
    r2 = r2_score(y, x)
    return r2 if r2 > 0 else 0

In [8]:
# Allocation and Revenue function
# Section 4.1
# TODO: How can we adjust sigma for the data?
noise_at_timestep = {}
def allocation_func(p_n, b_n, X_n, timestep, mu=0, sigma=0.1):
    """
    Allocation function for reals
    p_n: the price vector at previous timestep (t=n)
    b_n: the bid at previous timestep (in $ / model_score)
    X_n: the features available
    """
    global noise_at_timestep
    try:
        noise = noise_at_timestep[timestep]
    except KeyError:
        noise = np.random.normal(mu, sigma, X_n.shape)
        noise_at_timestep[timestep] = noise 
    return X_n + max(0, p_n - b_n) * noise
    
# B_epsilon parameters
B_EPS_STEP_SIZE = 1 # dollars
B_EPS_START = 0
B_EPS_END = 100

def _gain_func(p_n, X_n, Y_n, ml_func, random_state, bid):
    # Use cross-validation instead of holdout? ml_func uses CV to set params
    X_n_degraded = allocation_func(p_n, bid, X_n, timestep=0)
    _train, _test, y_n_train, y_n_test = train_test_split(X_n_degraded, Y_n, test_size=0.1, random_state=random_state)
    ml_func.fit(_train, y_n_train)
    return gain_score(y_n_test, ml_func.predict(_test))
    
def revenue_func(p_n, b_n, Y_n, context, random_state):
    """
    p_n: the price at the previous timestep (t=n)
    b_n: the bid at previous timestep (in $ / model_score)
    Y_n: the test Y data set
    """
    X_n = context.get('X_n')
    ml_func = context.get('ml_func')
    if X_n is None or ml_func is None:
        return None, None

    print('price: {}'.format(p_n))
    print('bid: {}'.format(b_n))
    
    _gain = partial(_gain_func, p_n, X_n, Y_n, ml_func, random_state)
    
    stopping_point = min(p_n, b_n)
    b_n_gain = _gain(stopping_point)
    print('b_n_gain: {}'.format(b_n_gain))
#     if b_n_gain == 0:
#         import pdb; pdb.set_trace()

    delta_revenue_list = []
    prev_bid = 0
    for bid in range(0, stopping_point, B_EPS_STEP_SIZE):
        other_gain = _gain(bid)
        revenue = (bid - prev_bid) * (b_n_gain - other_gain)
        # If revenue would be greater with a lower bid, no new revenue
        if (revenue < 0):
            revenue = 0
        prev_bid = bid
        delta_revenue_list.append(revenue)
    return sum(delta_revenue_list), np.cumsum(delta_revenue_list)


In [9]:
X_n = X_t1
X_n.shape

(400, 117)

In [10]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)
def test1(random_state=100):
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'ml_func': clf,
        'X_n': X_n,
    }
    b_n = 5
    p_n = b_n
    
    X_n_degraded = allocation_func(p_n, b_n, X_n, timestep=0)
    # assert X_n_degraded == X_n
    assert not np.any(X_n_degraded - X_n)
    
    for y_choice_idx in range(Y_t1.shape[0]):
        Y_n = Y_t1[y_choice_idx]
        revenue, cumulative_r = revenue_func(p_n, b_n, Y_n, context, random_state)
        print(revenue, cumulative_r)

test1()

price: 5
bid: 5
b_n_gain: 0.6828283019189709
2.18934392682 [ 0.          0.6828283   1.34470017  1.8814094   2.18934393]
price: 5
bid: 5
b_n_gain: 0.7341734278978391
0.722768700973 [ 0.          0.35832286  0.59955626  0.70606283  0.7227687 ]
price: 5
bid: 5
b_n_gain: 0.29584189517603254
0.562519868618 [ 0.          0.21912846  0.40564661  0.51557029  0.56251987]
price: 5
bid: 5
b_n_gain: 0.5702465874176466
1.05647196553 [ 0.          0.47519148  0.84587794  1.0323062   1.05647197]


In [11]:
def test2(random_state=100):
    # Why does LassoCV not work with extra features for Diabetes dataset?
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'ml_func': clf,
        'X_n': X_n,
    }
    feature_count = X_n.shape[1]
    b_n = 9
    p_n = 10
        
    y_choice_idx = 1
    Y_n = Y_t1[y_choice_idx]
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    for i in range(3):
        revenue, cumulative_revenue_list = revenue_func(p_n, b_n, Y_n, context, random_state)
        print(revenue, cumulative_revenue_list)
test2()

price: 10
bid: 9
b_n_gain: 0.7174675561151411
3.42805258651 [ 0.          0.65939321  1.27268677  1.83413909  2.33755274  2.77210737
  3.11372436  3.33825189  3.42805259]
price: 10
bid: 9
b_n_gain: 0.7174675561151411
3.42805258651 [ 0.          0.65939321  1.27268677  1.83413909  2.33755274  2.77210737
  3.11372436  3.33825189  3.42805259]
price: 10
bid: 9
b_n_gain: 0.7174675561151411
3.42805258651 [ 0.          0.65939321  1.27268677  1.83413909  2.33755274  2.77210737
  3.11372436  3.33825189  3.42805259]
price: 10
bid: 9
b_n_gain: 0.7174675561151411
3.42805258651 [ 0.          0.65939321  1.27268677  1.83413909  2.33755274  2.77210737
  3.11372436  3.33825189  3.42805259]
price: 10
bid: 9
b_n_gain: 0.7174675561151411
3.42805258651 [ 0.          0.65939321  1.27268677  1.83413909  2.33755274  2.77210737
  3.11372436  3.33825189  3.42805259]
price: 10
bid: 9
b_n_gain: 0.7174675561151411
3.42805258651 [ 0.          0.65939321  1.27268677  1.83413909  2.33755274  2.77210737
  3.11372436

In [12]:
def test3(feature_count=3, random_state=100):
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'ml_func': clf,
        'X_n': X_n[:, :feature_count],
    }
    b_n = 5
    p_n = 6
    Y_n = Y_t1[3]
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    revenue, cumulative = revenue_func(p_n, b_n, Y_n, context, random_state)
    print(revenue, cumulative)
test3()

price: 6
bid: 5


ValueError: operands could not be broadcast together with shapes (400,3) (400,117) 

In [ ]:
# Payment Division Function

In [ ]:
def payment_division_func(feature_indices, Y_n, M, ml_func, K):
    """
    S_n: the features allocated to buyer n
    Y_n: the test Y data set
    M: the number of sellers
    gain_func: the gain function
    K: the number of times to sample uniformly from S_n
    """
    for m in feature_indicesn:
        for k in range(K):
            sigma_k = np.random.permutation(feature_indices)
            #TODO: continue from line 4 of Algorithm 2
#             _train = get_original_columns(X_n_train, feature_idxs=feature_options[:idx+1])
#             _test = get_original_columns(X_n_test, feature_idxs=feature_options[:idx+1])
#             gain_func.fit(_train, y_n_train)
#             result = gain_score(y_n_test, gain_func.predict(_test))
#             gain = gain_func(Y_n, )

In [ ]:
# Price Update Function

In [ ]:
def draw(weights):
    """pick the index of a weight by normalizing to a probability distribution and drawing according to that distribution"""
    return np.random.choice(range(len(weights)), p = preprocessing.normalize(weights.reshape(1, -1), norm='l1').ravel())

B_eps = np.arange(B_EPS_START, B_EPS_END, B_EPS_STEP_SIZE)
weights = np.ones(len(B_eps))
def multi_weights(B_eps, reward, learningRate = 0.1):
    global weights
    print(weights)
    for i in range(len(weights)):
        weights[i] *= (1 + learningRate * reward(B_eps[i]))
    return B_eps[draw(weights)]

def price_update_func(b_n, Y_n, delta=0.1, random_state=42):
    """
    b_n: the current bid
    Y_n: the test Y data set
    delta: the learning rate for the multiplicative weights algorithm
    """
    clf = linear_model.LassoCV(cv=5, random_state=100)
    context = {
        'ml_func': clf,
        'X_n': X_n,
    }
    def reward(price):
        revenue, cumulative_r = revenue_func(price, b_n, Y_n, context, random_state)
        return revenue / B_EPS_END
    return multi_weights(B_eps, reward, delta)

price_update_func(5, Y_t1[3])
